In [3]:
import threading
import DobotDllType as dType
import numpy as np
import math
import time

In [4]:
CON_STR = {
    dType.DobotConnect.DobotConnect_NoError:  "DobotConnect_NoError",
    dType.DobotConnect.DobotConnect_NotFound: "DobotConnect_NotFound",
    dType.DobotConnect.DobotConnect_Occupied: "DobotConnect_Occupied"}

In [5]:
#将dll读取到内存中并获取对应的CDLL实例
#Load Dll and get the CDLL object
api = dType.load()
#建立与dobot的连接
#Connect Dobot
state = dType.ConnectDobot(api, "", 115200)[0]
print("Connect status:",CON_STR[state])

您用的dll是64位，为了顺利运行，请保证您的python环境也是64位
python环境是： ('64bit', 'WindowsPE')
Connect status: DobotConnect_NoError


In [6]:
dType.SetQueuedCmdClear(api)

[0]

In [5]:
dType.SetPTPJointParams(api,200,200,200,200,200,200,200,200,0)
dType.SetPTPCoordinateParams(api,200,200,200,200,0)
dType.SetPTPJumpParams(api, 10, 200,0)
dType.SetPTPCommonParams(api, 100, 100,0)
moveX=0;moveY=0;moveZ=10;moveFlag=-1
pos = dType.GetPose(api)
x = pos[0]
y = pos[1]
z = pos[2]
rHead = pos[3]

In [6]:
moveFlag *= -1
for i in range(5):
    dType.SetPTPCmd(api, 2, x+moveX, y+moveY, z+moveZ, rHead, 1)
    moveX += 10 * moveFlag
    dType.SetPTPCmd(api, 2, x+moveX, y+moveY, z+moveZ, rHead, 1)
    dType.SetPTPCmd(api, 2, x+moveX, y+moveY, z, rHead, 1)

In [ ]:
pos = dType.GetPose(api)
x = pos[0]
y = pos[1]
z = pos[2]

print(x,y,z)

In [ ]:
pos = dType.GetPose(api)
x = pos[0]
y = pos[1]
z = pos[2]

print(x,y,z)

In [15]:
# prev_pose = dType.GetPose(api)
# while(True):
#     time.sleep(0.5)
#     pose = dType.GetPose(api)
#     if prev_pose == pose:
#         continue
#     prev_pose = pose
#     print(pose)

In [7]:
pickupLoc = {}
refLoc = {}
homeLoc = None
globZ = None

boardPos = {}

In [8]:
locList = ["Z", "Pickup Loc 1", "Pickup Loc 2", "Pickup Loc 3", "Home", "Ref 1", "Ref 2"]
def setCoordinates(locList = locList):
    global pickupLoc, refLoc, homeLoc, globZ
    prev_pose = dType.GetPose(api)
    allLoc = []
    cnt = 0
    while(True):
        time.sleep(0.5)
        pose = dType.GetPose(api)
        if prev_pose == pose:
            continue
        prev_pose = pose
        allLoc.append(pose)
        print(f"Location set: {locList[cnt]}")
        cnt += 1
        if len(allLoc) == 7:
            break


    globZ = allLoc[0][2]
    pickupLoc[1] = (allLoc[1][0], allLoc[1][1], globZ) 
    pickupLoc[2] = (allLoc[2][0], allLoc[2][1], globZ) 
    pickupLoc[3] = (allLoc[3][0], allLoc[3][1], globZ)

    homeLoc = (allLoc[4][0], allLoc[4][1], allLoc[4][2]) 
    refLoc[1] = (allLoc[5][0], allLoc[5][1]) 
    refLoc[2] = (allLoc[6][1], allLoc[6][1])

def setCoordinatesHome():
    global homeLoc
    prev_pose = dType.GetPose(api)
    while(True):
        time.sleep(0.5)
        pose = dType.GetPose(api)
        if prev_pose == pose:
            continue
        prev_pose = pose
        homeLoc = (pose[0], pose[1], pose[2])
        print(f"Location set: Home")
        break


def pick(location):
    x = location[0]
    y = location[1]

    current_pose = dType.GetPose(api)
    dType.SetPTPCmdEx(api, 2, x,  y,  globZ+30, current_pose[3], 1)
    current_pose = dType.GetPose(api)
    dType.SetPTPCmdEx(api, 2, x,  y,  globZ-5, current_pose[3], 1)
    dType.SetEndEffectorSuctionCupEx(api, 1, 1)
    dType.dSleep(1000)
    current_pose = dType.GetPose(api)
    dType.SetPTPCmdEx(api, 2, homeLoc[0],  homeLoc[1],  homeLoc[2], current_pose[3], 1)

def drop(location):
    x = location[0]
    y = location[1]

    current_pose = dType.GetPose(api)
    dType.SetPTPCmdEx(api, 2, x,  y,  globZ+30, current_pose[3], 1)
    current_pose = dType.GetPose(api)
    dType.SetPTPCmdEx(api, 2, x,  y,  globZ+5, current_pose[3], 1)
    dType.SetEndEffectorSuctionCupEx(api, 0, 1)
    dType.dSleep(1000)
    current_pose = dType.GetPose(api)
    dType.SetPTPCmdEx(api, 2, homeLoc[0],  homeLoc[1],  homeLoc[2], current_pose[3], 1)

def playPiece(iteration, location):
    pickupLocation = pickupLoc[iteration]
    pick(pickupLocation)

    finalPosition = boardPos[location]
    drop(finalPosition)


In [13]:
setCoordinates()

Location set: Z
Location set: Pickup Loc 1
Location set: Pickup Loc 2
Location set: Pickup Loc 3
Location set: Home
Location set: Ref 1
Location set: Ref 2


In [10]:
setCoordinatesHome()

Location set: Home


In [73]:
globZ, pickupLoc, refLoc, homeLoc

(-41.913856506347656,
 {1: (216.90786743164062, 82.30638885498047, -41.913856506347656),
  2: (225.9152374267578, 54.1270751953125, -41.913856506347656),
  3: (235.9609832763672, 24.176097869873047, -41.913856506347656)},
 {1: (285.6653137207031, -91.07048034667969),
  2: (31.713298797607422, 31.713298797607422)},
 (224.08053588867188, -22.645063400268555, 123.52239990234375))

In [14]:
pick(pickupLoc[1])

In [57]:
dType.SetEndEffectorSuctionCupEx(api, 0, 1)

In [15]:
drop(pickupLoc[2])